In [12]:
%cd /src/code
from typing import Dict, Any, List
import numpy as np
import pandas as pd
import kaggle
from kaggle.api.kaggle_api_extended import KaggleApi
import subprocess
import hashlib
import os

#TODO: set random state

pd.set_option('display.max_columns', None)
api = KaggleApi()
api.authenticate()

target_col: str = 'failure_prone'
submission_id: str = 'U8AXLZ'

/src/code


# EDA

In [3]:
X_train = pd.read_csv('../assets/X_train.csv', index_col='id')
y_train = pd.read_csv('../assets/y_train.csv', index_col='id')[target_col].astype(int)

X_train.head()

,additions,additions_avg,additions_max,avg_play_size,avg_task_size,change_set_avg,change_set_max,code_churn_avg,code_churn_count,code_churn_max,commits_count,contributors_count,deletions,deletions_avg,deletions_max,highest_contributor_experience,hunks_median,lines_blank,lines_code,lines_comment,minor_contributors_count,num_authorized_key,num_block_error_handling,num_blocks,num_commands,num_conditions,num_decisions,num_deprecated_keywords,num_deprecated_modules,num_distinct_modules,num_external_modules,num_fact_modules,num_file_exists,num_file_mode,num_file_modules,num_filters,num_ignore_errors,num_import_playbook,num_import_role,num_import_tasks,num_include,num_include_role,num_include_tasks,num_include_vars,num_keys,num_lookups,num_loops,num_math_operations,num_names_with_vars,num_parameters,num_paths,num_plays,num_prompts,num_regex,num_roles,num_suspicious_comments,num_tasks,num_tokens,num_unique_names,num_uri,num_vars,text_entropy,delta_avg_play_size,delta_avg_task_size,delta_lines_blank,delta_lines_code,delta_lines_comment,delta_num_authorized_key,delta_num_block_error_handling,delta_num_blocks,delta_num_commands,delta_num_conditions,delta_num_decisions,delta_num_deprecated_keywords,delta_num_deprecated_modules,delta_num_distinct_modules,delta_num_external_modules,delta_num_fact_modules,delta_num_file_exists,delta_num_file_mode,delta_num_file_modules,delta_num_filters,delta_num_ignore_errors,delta_num_import_playbook,delta_num_import_role,delta_num_import_tasks,delta_num_include,delta_num_include_role,delta_num_include_tasks,delta_num_include_vars,delta_num_keys,delta_num_lookups,delta_num_loops,delta_num_math_operations,delta_num_names_with_vars,delta_num_parameters,delta_num_paths,delta_num_plays,delta_num_prompts,delta_num_regex,delta_num_roles,delta_num_suspicious_comments,delta_num_tasks,delta_num_tokens,delta_num_unique_names,delta_num_uri,delta_num_vars,delta_text_entropy
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,0,0,0,0.0,10.0,2,27,0,0,0,0,0,0,0,0,0.0,0.0,8.0,87.0,5.0,0,0.0,0.0,0.0,5.0,9.0,5.0,1.0,0.0,9.0,0.0,0.0,0.0,2.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,51.0,0.0,5.0,0.0,1.0,17.0,5.0,0.0,0.0,0.0,0.0,0.0,9.0,293.0,11.0,0.0,1.0,6.39,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,0,0,0.0,6.0,1,1,0,0,0,0,0,0,0,0,0.0,0.0,24.0,103.0,59.0,0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,16.0,0.0,0.0,0.0,1.0,1.0,0.0,3.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,87.0,0.0,1.0,0.0,0.0,25.0,6.0,0.0,0.0,0.0,0.0,0.0,16.0,238.0,17.0,0.0,3.0,6.59,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,0,0,0.0,4.0,2,12,0,0,0,0,0,0,0,0,0.0,0.0,1.0,8.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,8.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,22.0,2.0,0.0,1.0,4.10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,0,0,0.0,6.0,1,4,0,0,0,0,0,0,0,0,0.0,0.0,5.0,33.0,0.0,0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,28.0,0.0,1.0,0.0,1.0,12.0,1.0,0.0,0.0,0.0,0.0,0.0,6.0,109.0,8.0,0.0,1.0,5.51,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,0,0,0.0,4.0,1,2,0,0,0,0,0,0,0,0,0.0,0.0,1.0,9.0,12.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0,3.0,2.0,0.0,0.0,0.0,0.0,0.0,2.0,24.0,2.0,0.0,0.0,4.14,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.

In [4]:
X_train.shape

(159090, 108)

In [5]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 159090 entries, 0 to 159089
Columns: 108 entries, additions to delta_text_entropy
dtypes: float64(94), int64(14)
memory usage: 132.3 MB


In [6]:
print(X_train.info())
X_train.describe()

<class 'pandas.core.frame.DataFrame'>
Index: 159090 entries, 0 to 159089
Columns: 108 entries, additions to delta_text_entropy
dtypes: float64(94), int64(14)
memory usage: 132.3 MB
None


,additions,additions_avg,additions_max,avg_play_size,avg_task_size,change_set_avg,change_set_max,code_churn_avg,code_churn_count,code_churn_max,commits_count,contributors_count,deletions,deletions_avg,deletions_max,highest_contributor_experience,hunks_median,lines_blank,lines_code,lines_comment,minor_contributors_count,num_authorized_key,num_block_error_handling,num_blocks,num_commands,num_conditions,num_decisions,num_deprecated_keywords,num_deprecated_modules,num_distinct_modules,num_external_modules,num_fact_modules,num_file_exists,num_file_mode,num_file_modules,num_filters,num_ignore_errors,num_import_playbook,num_import_role,num_import_tasks,num_include,num_include_role,num_include_tasks,num_include_vars,num_keys,num_lookups,num_loops,num_math_operations,num_names_with_vars,num_parameters,num_paths,num_plays,num_prompts,num_regex,num_roles,num_suspicious_comments,num_tasks,num_tokens,num_unique_names,num_uri,num_vars,text_entropy,delta_avg_play_size,delta_avg_task_size,delta_lines_blank,delta_lines_code,delta_lines_comment,delta_num_authorized_key,delta_num_block_error_handling,delta_num_blocks,delta_num_commands,delta_num_conditions,delta_num_decisions,delta_num_deprecated_keywords,delta_num_deprecated_modules,delta_num_distinct_modules,delta_num_external_modules,delta_num_fact_modules,delta_num_file_exists,delta_num_file_mode,delta_num_file_modules,delta_num_filters,delta_num_ignore_errors,delta_num_import_playbook,delta_num_import_role,delta_num_import_tasks,delta_num_include,delta_num_include_role,delta_num_include_tasks,delta_num_include_vars,delta_num_keys,delta_num_lookups,delta_num_loops,delta_num_math_operations,delta_num_names_with_vars,delta_num_parameters,delta_num_paths,delta_num_plays,delta_num_prompts,delta_num_regex,delta_num_roles,delta_num_suspicious_comments,delta_num_tasks,delta_num_tokens,delta_num_unique_names,delta_num_uri,delta_num_vars,delta_text_entropy
count,159090.000000,159090.000000,159090.000000,159090.000000,159090.000000,159090.000000,159090.000000,159090.000000,159090.000000,159090.000000,159090.000000,159090.000000,159090.000000,159090.000000,159090.000000,159090.000000,159090.000000,159090.000000,159090.000000,159090.000000,159090.000000,159090.000000,159090.000000,159090.000000,159090.000000,159090.000000,159090.000000,159090.000000,159090.000000,159090.000000,159090.000000,159090.000000,159090.000000,159090.000000,159090.000000,159090.000000,159090.000000,159090.000000,159090.000000,159090.000000,159090.000000,159090.000000,159090.000000,159090.000000,159090.000000,159090.000000,159090.000000,159090.0,159090.000000,159090.000000,159090.000000,159090.000000,159090.000000,159090.000000,159090.000000,159090.000000,159090.000000,159090.000000,159090.000000,159090.000000,159090.000000,159090.000000,159090.000000,159090.000000,159090.000000,159090.000000,159090.000000,159090.000000,159090.000000,159090.000000,159090.000000,159090.000000,159090.000000,159090.000000,159090.000000,159090.000000,159090.000000,159090.000000,159090.00000,159090.000000,159090.000000,159090.000000,159090.000000,159090.000000,159090.000000,159090.000000,159090.000000,159090.000000,159090.000000,159090.000000,159090.000000,159090.000000,159090.000000,159090.0,159090.000000,159090.000000,159090.000000,159090.000000,159090.000000,159090.000000,159090.000000,159090.000000,159090.000000,159090.000000,159090.000000,159090.000000,159090.000000,159090.000000
mean,2.499761,1.297800,1.885650,1.347525,15.022539,2.331096,17.088799,0.848752,1.439368,1.440072,0.237601,0.146125,1.060393,0.446760,0.766478,10.062212,0.209699,5.565022,38.490469,5.388526,0.010591,0.002368,0.005172,0.127135,0.826643,2.309171,1.060175,0.107637,0.001098,4.525189,0.416568,0.007873,0.004953,0.348019,0.395093,1.693051,0.387970,0.008230,0.008998,0.059486,0.177359,0.124411,0.189119,0.058140,29.095852,0.048543,0.682085,0.0,0.534381,7.421218,1.634415,0.101617,0.052280,0.103403,0.092530,0.037669,4.941783,127.998317,5.204056,0.020711,1.156392,4.767544,-0.005813,0.

In [7]:
print(y_train.info())
y_train.describe()

<class 'pandas.core.series.Series'>
Index: 159090 entries, 0 to 159089
Series name: failure_prone
Non-Null Count   Dtype
--------------   -----
159090 non-null  int64
dtypes: int64(1)
memory usage: 2.4 MB
None


count    159090.000000
mean          0.060582
std           0.238563
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max           1.000000
Name: failure_prone, dtype: float64

In [8]:
y_train.head()

id
0    0
1    0
2    0
3    0
4    0
Name: failure_prone, dtype: int64

In [9]:
y_train.isna().sum()

0

In [10]:
y_train.value_counts()

failure_prone
0    149452
1      9638
Name: count, dtype: int64

# Hyperparameter tuning

use 10 fold

In [17]:

hyperparameters_trials: List[Dict[str, Any]] = [
    {
        'model': 'RandomForestClassifier',
        'framework': 'scikit-learn~=1.4',
    },
]

hyperparameters: Dict[str, Any] = hyperparameters_trials[0]  # TODO

'''
* you can define conditional spaces using `hp.choice`, and inside the choices go the entire hyperparameter set that is dependent; this leads to a lot of duplication; conditional parameters will only appear in some of the options;
[example](https://stackoverflow.com/questions/43859465/problems-setting-up-conditional-search-space-in-hyperopt)
* you can have "fixed" hyperparameters, that won't be optimized, just by adding a key like `'model': LogisticRegression`
'''


'''
#https://github.com/LeonardoSanBenitez/tutorial-mlflow/blob/main/code/2.0%20-%20AutoML%20with%20hyperopt.ipynb
%%time
def objective(max_depth: float) -> dict:
    with mlflow.start_run(nested=True):
        mlflow.log_params({'max_depth': max_depth})
        
        # Train model
        model = RandomForestRegressor(n_estimators=1, max_depth=int(max_depth))
        model.fit(X_train, y_train)
        mlflow.sklearn.log_model(
            sk_model=model,
            artifact_path="model",
            signature=mlflow.models.infer_signature(X_train, model.predict(X_train)),
            input_example=X_train,
        )
        
        # Evaluate model
        y_pred = model.predict(X_test)
        mae = mean_absolute_error(y_test, y_pred)
        mlflow.log_metric("mean_absolute_error_test", mean_absolute_error(y_test, y_pred))
        return {'loss': -mae, 'status': STATUS_OK}

mlflow.set_tracking_uri('http://server:5000')
mlflow.set_experiment("hyperopt2")
with mlflow.start_run(description='Parent run for hyperopt over max_depth', nested=True):
    argmin = fmin(
      fn=objective,
      space=hp.quniform('max_depth', low=1, high=300, q=1), # space of values to test
      algo=tpe.suggest,
      max_evals=5,
    )
    
    print("Best value found: ", argmin)
'''

'\n#https://github.com/LeonardoSanBenitez/tutorial-mlflow/blob/main/code/2.0%20-%20AutoML%20with%20hyperopt.ipynb\n%%time\ndef objective(max_depth: float) -> dict:\n    with mlflow.start_run(nested=True):\n        mlflow.log_params({\'max_depth\': max_depth})\n        \n        # Train model\n        model = RandomForestRegressor(n_estimators=1, max_depth=int(max_depth))\n        model.fit(X_train, y_train)\n        mlflow.sklearn.log_model(\n            sk_model=model,\n            artifact_path="model",\n            signature=mlflow.models.infer_signature(X_train, model.predict(X_train)),\n            input_example=X_train,\n        )\n        \n        # Evaluate model\n        y_pred = model.predict(X_test)\n        mae = mean_absolute_error(y_test, y_pred)\n        mlflow.log_metric("mean_absolute_error_test", mean_absolute_error(y_test, y_pred))\n        return {\'loss\': -mae, \'status\': STATUS_OK}\n\nmlflow.set_tracking_uri(\'http://server:5000\')\nmlflow.set_experiment("hyper

In [18]:
'''
# Scikitlearn example of stratified k-fold cross-validation with an imbalanced dataset (Source: ML mastery)

from sklearn.datasets import make_classification
from sklearn.model_selection import StratifiedKFold
# generate 2 class dataset
X, y = make_classification(n_samples=1000, n_classes=2, weights=[0.99, 0.01], flip_y=0, random_state=1)
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)
# enumerate the splits and summarize the distributions
for train_ix, test_ix in kfold.split(X, y):
	# select rows
	train_X, test_X = X[train_ix], X[test_ix]
	train_y, test_y = y[train_ix], y[test_ix]
	# summarize train and test composition
	train_0, train_1 = len(train_y[train_y==0]), len(train_y[train_y==1])
	test_0, test_1 = len(test_y[test_y==0]), len(test_y[test_y==1])
	print('>Train: 0=%d, 1=%d, Test: 0=%d, 1=%d' % (train_0, train_1, test_0, test_1)

'''

"\n# Scikitlearn example of stratified k-fold cross-validation with an imbalanced dataset (Source: ML mastery)\n\nfrom sklearn.datasets import make_classification\nfrom sklearn.model_selection import StratifiedKFold\n# generate 2 class dataset\nX, y = make_classification(n_samples=1000, n_classes=2, weights=[0.99, 0.01], flip_y=0, random_state=1)\nkfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)\n# enumerate the splits and summarize the distributions\nfor train_ix, test_ix in kfold.split(X, y):\n\t# select rows\n\ttrain_X, test_X = X[train_ix], X[test_ix]\n\ttrain_y, test_y = y[train_ix], y[test_ix]\n\t# summarize train and test composition\n\ttrain_0, train_1 = len(train_y[train_y==0]), len(train_y[train_y==1])\n\ttest_0, test_1 = len(test_y[test_y==0]), len(test_y[test_y==1])\n\tprint('>Train: 0=%d, 1=%d, Test: 0=%d, 1=%d' % (train_0, train_1, test_0, test_1)\n\n"

# Train final model

In [19]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

model = RandomForestClassifier(n_estimators=5, max_depth=10, random_state=0)
model.fit(X_train, y_train)
#print(model.predict([[0, 0, 0, 0]]))

RandomForestClassifier(max_depth=10, n_estimators=5, random_state=0)

In [20]:
type(model)

sklearn.ensemble._forest.RandomForestClassifier

False

True

# Submit best model
Expects a variable `model`, with a method `predict` that follows a sklean-like interface, and a dictionary `hyperparameters`

In [25]:
assert model is not None
assert hasattr(model, 'predict')

X_test = pd.read_csv('../assets/X_test.csv', index_col='id')
assert X_test.shape[1] == X_train.shape[1]
assert all(X_test.columns == X_train.columns)

y_test_hat = model.predict(X_test)
assert len(y_test_hat.shape) == 1
assert y_test_hat.shape[0] == X_test.shape[0]

assert y_test_hat.min() == 0
assert y_test_hat.max() == 1
assert y_test_hat.dtype == np.dtype('int64')

In [40]:
# Submission params
competition = 'ppcu-data-mining-and-machine-learning-2024'
file_name = f'../assets/y_test_submission_{submission_id}.csv'
message = f"test trained submission"
message += f"; commit_hash: {subprocess.check_output(['git', 'rev-parse', 'HEAD']).decode('ascii').strip()}"
message += f"; hyperparameters_hash: {hashlib.md5(str(hyperparameters).encode()).hexdigest()}"

# Run inference
if os.path.exists(file_name):
    os.remove(file_name)
pd.DataFrame(y_test_hat, index=X_test.index, columns=["failure_prone"]).to_csv(file_name, index=True)
assert os.path.exists(file_name)

# Submit the file to the competition
#api.competition_submit(file_name, message, competition)


100%|██████████| 522k/522k [00:01<00:00, 533kB/s]


Successfully submitted to PPCU - Data mining and machine learning 2024